In [1]:
!pip install pandas pgmpy

In [2]:
import numpy as np
import csv
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import TabularCPD


In [3]:
# Define the structure of the Bayesian Network
model = BayesianModel([
    ('Age', 'HeartDisease'),
    ('Cholesterol', 'HeartDisease'),
    ('BloodPressure', 'HeartDisease'),
    ('MaxHeartRate', 'HeartDisease'),
    ('ExerciseAngina', 'HeartDisease'),
])

In [4]:
# CPD for Age
cpd_age = TabularCPD(variable='Age', variable_card=3,
                     values=[[0.5], [0.3], [0.2]])  # Example probabilities for age categories

# CPD for Cholesterol
cpd_cholesterol = TabularCPD(variable='Cholesterol', variable_card=3,
                             values=[[0.4], [0.4], [0.2]])  # Example probabilities for cholesterol

# CPD for Blood Pressure
cpd_blood_pressure = TabularCPD(variable='BloodPressure', variable_card=3,
                                 values=[[0.3], [0.4], [0.3]])  # Example probabilities

# CPD for Max Heart Rate
cpd_max_heart_rate = TabularCPD(variable='MaxHeartRate', variable_card=3,
                                 values=[[0.5], [0.4], [0.1]])  # Example probabilities

# CPD for Exercise Angina
cpd_exercise_angina = TabularCPD(variable='ExerciseAngina', variable_card=2,
                                  values=[[0.7], [0.3]])  # Example probabilities

# Correct CPD for Heart Disease
import numpy as np
cpd_heart_disease = TabularCPD(
    variable='HeartDisease',
    variable_card=2,
    values=[
        np.random.rand(162),  # No Heart Disease
        np.random.rand(162)   # Heart Disease
    ],
    evidence=['Age', 'Cholesterol', 'BloodPressure', 'MaxHeartRate', 'ExerciseAngina'],
    evidence_card=[3, 3, 3, 3, 2]
)

# Normalize the values so that they sum to 1 for each column
cpd_heart_disease.normalize()

# Add all CPDs to the model
model.add_cpds(cpd_age, cpd_cholesterol, cpd_blood_pressure, cpd_max_heart_rate, cpd_exercise_angina, cpd_heart_disease)

# Check if the model is valid
assert model.check_model()

# Perform inference on the model
infer = VariableElimination(model)

# Example query: What is the probability of having heart disease given some evidence?
q = infer.query(variables=['HeartDisease'], evidence={'Age': 1, 'Cholesterol': 2, 'BloodPressure': 1})
print(q)


+-----------------+---------------------+
| HeartDisease    |   phi(HeartDisease) |
+=================+=====================+
| HeartDisease(0) |              0.5447 |
+-----------------+---------------------+
| HeartDisease(1) |              0.4553 |
+-----------------+---------------------+
